In [1]:
from z3 import Solver, ForAll, Exists, Not, And, Or, Implies
from meyer.util.z3py_util import const, consts
from meyer.util.z3py_set import set, sets, included
from meyer.program import U, prog, progs, conclude
from meyer.equivalence import eq, eq_set, eq_pre, eq_post
from meyer.feasibility import feasible
from meyer.basic_constructs import Choi, Comp, Rest, Corest
from meyer.refinement import is_ref_of
from meyer.special_programs import Fail, Havoc, Skip, total

s = Solver()

dom(C:p)⊆C かつ ran(p＼C) ⊆ C が必要？

equalではないが，equivalence

In [2]:
title = "P26 (p \ C) = (p; (C: Skip))"
p = prog(s, 'p')
C = set('C', U)
lhs = Corest(p, C)
rhs = Comp(p, Rest(C, Skip()))
conclude(s, eq(lhs, rhs), title)

P26 (p \ C) = (p; (C: Skip))
Universe = U, has 3 element(s)
Holds: unsat 



P27 U=Intのとき反例を生成

In [3]:
title = "P27 (p1 ∪ p2) \ C = (p1 \ C) ∪ (p2 \ C)"
p1, p2 = progs(s, 'p1 p2')
C = set('C', U)
s.add(eq_pre(p1, p2)) # Additional assumption
# s.add(feasible(p1, p2)) # makes counter example
lhs = Corest(Choi(p1, p2), C)
rhs = Choi(Corest(p1, C), Corest(p2, C))
conclude(s, eq(lhs, rhs), title)

P27 (p1 ∪ p2) \ C = (p1 \ C) ∪ (p2 \ C)
Universe = U, has 3 element(s)
Holds: unsat 



In [4]:
title = "P28 (p1 ; p2) \ C = p1 ; (p2 \ C)"
p1, p2 = progs(s, 'p1 p2')
C = set('C', U)
lhs = Corest(Comp(p1, p2), C)
rhs = Comp(p1, Corest(p2, C))
conclude(s, eq(lhs, rhs), title)

P28 (p1 ; p2) \ C = p1 ; (p2 \ C)
Universe = U, has 3 element(s)
Holds: unsat 



P29 U=A,B,Cのとき反例を生成<br>
Cではなくpなのか？C:pの可能性はないか？

In [7]:
title = "P29 (p \ C) ⊆ p but (p \ C) ⊆ C"
p = prog(s, 'p')
C = set('C', U)
s.add(feasible(p))
# Additional assumption
x, y = consts('x y', U)
s.add(ForAll(x, Implies(p.pre(x), Exists(y, And(p.post(x,y), C.has(y))))))
conclude(s, is_ref_of(Corest(p, C), Rest(C, p)), title)

P29 (p \ C) ⊆ p but (p \ C) ⊆ C
Universe = U, has 3 element(s)
Holds: unsat 



In [6]:
title = "P30 If D ⊆ C then (p \ D) ⊆ (p \ C)"
p = prog(s, 'p')
D, C = sets('D C', U)
s.add(included(D, C))
s.add(feasible(p))
# Additional assumption
x, y, z = consts('x y z', U)
s.add(ForAll(x, Implies(
    p.pre(x), 
    And(
        Exists(y, And(p.post(x,y), C.has(y))),
        Exists(z, And(p.post(x,z), D.has(z)))
    )
)))
conclude(s, is_ref_of(Corest(p, D), Corest(p, C)), title)

P30 If D ⊆ C then (p \ D) ⊆ (p \ C)
Universe = U, has 3 element(s)
Holds: unsat 

